In [2]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.pipeline import Pipeline ,make_pipeline
from sklearn import impute
from sklearn import compose
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, plot_confusion_matrix,roc_auc_score
from sklearn import set_config
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler,Normalizer,PowerTransformer,QuantileTransformer, RobustScaler,StandardScaler,LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.tree          import DecisionTreeClassifier
from sklearn.linear_model  import LogisticRegression    
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn
set_config(display='diagram') # Useful for display the pipeline
print("Pandas  ", pd.__version__)
print("Sklearn ", sklearn.__version__) # Try to use 0.24



Pandas   1.2.4
Sklearn  0.24.2


In [4]:
data = pd.read_csv('./data/feature_eng.csv')
data.head(5)

,id,user,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,speed_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target,acc_gyro,Lin_speed
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.180379,0.050413,0.056351,9.758895,51.199707,NaN,Still,0.000083,0.003784
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,14.679876,0.999981,0.999999,7.707437,82.409890,89.065143,Car,0.036325,12.906844
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.180379,0.610456,0.610456,9.804817,55.501802,NaN,Still,0.000931,0.006153
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,14.679876,0.998112,0.998112,7.659674,95.664309,87.470377,Car,0.036372,21.848949
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,14.679876,0.322242,0.378193,8.965621,156.795909,89.770732,Car,0.059035,24.343749


# Create test data 

In [5]:
# create test data from some user 
df=data.copy()
user1=df[df['user']=='Pierpaolo']
user2=df[df['user']=='IvanHeibi']
user3=df[df['user']=='AndreaCarpineti']
user4=df[df['user']=='Elena']
users=[user1,user2,user3,user4]
df_test=pd.concat(users)
df_test.head(5)

,id,user,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,speed_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target,acc_gyro,Lin_speed
33,19919,Pierpaolo,0.615399,1.283204,0.655962,0.778458,14.028126,1.721314,1.120206,0.955202,0.955202,9.764513,36.378866,77.064227,Walking,0.587830,1.928226
54,19494,Pierpaolo,0.447802,0.361961,0.116564,0.334574,9.963264,0.958200,5.368554,0.983561,0.989576,8.712177,77.496043,74.178611,Bus,0.440440,5.144149
102,20104,Pierpaolo,0.069503,0.070605,0.048140,0.078626,9.187340,0.868719,1.120206,0.787570,0.802811,9.012243,91.951478,6.020600,Walking,0.054738,0.973144
121,19436,Pierpaolo,0.045041,0.451773,0.027367,0.038644,9.563967,0.958079,5.368554,0.993876,0.996117,8.246220,70.436446,72.559600,Bus,0.044765,5.143499
172,20245,Pierpaolo,1.734294,4.272022,1.400249,1.331566,17.743662,8.823918,1.120206,0.985658,0.993053,3.332163,90.118737,73.702289,Walking,1.709421,9.884602


## Subtract test data from data 

In [6]:
df_train = df.merge(df_test, how='left', indicator=True)
df_train = df_train[df_train['_merge'] == 'left_only']
df_train=df_train.drop('_merge',axis=1)
df_train.head(5)

,id,user,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,speed_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target,acc_gyro,Lin_speed
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.180379,0.050413,0.056351,9.758895,51.199707,NaN,Still,0.000083,0.003784
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,14.679876,0.999981,0.999999,7.707437,82.409890,89.065143,Car,0.036325,12.906844
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.180379,0.610456,0.610456,9.804817,55.501802,NaN,Still,0.000931,0.006153
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,14.679876,0.998112,0.998112,7.659674,95.664309,87.470377,Car,0.036372,21.848949
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,14.679876,0.322242,0.378193,8.965621,156.795909,89.770732,Car,0.059035,24.343749


In [7]:
print(df_train.shape)
print(df_test.shape)

(4985, 17)
(908, 17)


## drop id ,time, user columns

In [8]:
unimp    = ['id','user']
df_train = df_train.drop(unimp,axis=1)
df_test  = df_test.drop(unimp,axis=1)

In [9]:
df_train=df_train.drop('speed_mean',axis=1)
df_test=df_test.drop('speed_mean',axis=1)

In [10]:
print(df_train.shape)
print(df_test.shape)

(4985, 14)
(908, 14)


In [11]:
X=df_train.drop('target',axis=1)
y=df_train.target
X_test=df_test.drop('target',axis=1)
y_test=df_test.target

# Pipeline 

In [12]:
num_attribs=df_train.drop('target',axis=1).columns.to_list()
num_pip=Pipeline([('imputer',impute.SimpleImputer(strategy='median')),
                  ('scalar',MinMaxScaler()),
               ])  # ('PCA',PCA(n_components=12))
preprocessor=compose.ColumnTransformer([
    ('num',num_pip,num_attribs)
])

In [13]:
# pipeline for model
classifier_models = {
"DecisionTreeClassifier": DecisionTreeClassifier(),
'KNeighborsClassifier':KNeighborsClassifier(),
'LogisticRegression':LogisticRegression(),
"RandomForestClassifier":RandomForestClassifier(),
"GradientBoostingClassifier":GradientBoostingClassifier(),
"XGBClassifier":XGBClassifier(),
"LGBMClassifier":LGBMClassifier(),
'GaussianNB':GaussianNB(),

}
# make pipline with  preprocessing 
classifier_models = {name: make_pipeline(preprocessor, model) for name, model in classifier_models.items()}
classifier_models["GradientBoostingClassifier"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scalar',
                                                                   MinMaxScaler())]),
                                                  ['android_sensor_gyroscope_mean',
                                                   'android_sensor_accelerometer_std',
                                                   'android_sensor_gyroscope_std',
                                                   'android_sensor_gyroscope_uncalibrated_mean',
                                                   'android_sensor_accelerometer_max',
                                                   'android_sensor_linear_acceleration_mean',
                                                   'android_sensor_rotation_vector_mean',
                                                   'android_sensor_rotation_vector_max',
                                                   'android_sensor_accelerometer_min',
                                                   'android_sensor_magnetic_field_uncalibrated_min',
                                                   'sound_min', 'acc_gyro',
                                                   'Lin_speed'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [14]:


x_train, x_val, y_train, y_val = train_test_split(X, y,test_size=0.4,stratify = y,random_state=10 )
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    valid_pred = model.predict(x_val)
    results = results.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_val, valid_pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_val, valid_pred)*100,
                    
                              "Time":     total_time},
                              ignore_index=True)
results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


[15:36:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,Bal Acc.,Time
1,LGBMClassifier,96.840522,96.798966,0.965629
2,XGBClassifier,96.188566,96.225113,1.793139
3,GradientBoostingClassifier,95.436309,95.096696,6.033553
4,RandomForestClassifier,95.035105,94.929522,0.650320
5,DecisionTreeClassifier,94.282849,94.541477,0.042946
6,KNeighborsClassifier,87.061184,87.473061,0.018347
7,LogisticRegression,68.706118,70.018030,0.305470
8,GaussianNB,63.289870,65.032958,0.014976


# Best model for test data  

In [15]:
best_model=LGBMClassifier()
name='LGBMClassifier'

In [16]:
clf=Pipeline([('preprocess',preprocessor),('classification',best_model)])

In [17]:

test_result = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time = time.time()
    model.fit(X, y)
    total_time = time.time() - start_time
    pred = model.predict(X_test)
    test_result = test_result.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_test, pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
test_result_ord = test_result.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
test_result_ord.index += 1 
test_result_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


[15:37:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,Bal Acc.,Time
1,LGBMClassifier,83.810573,93.101400,0.792724
2,XGBClassifier,76.211454,83.216316,2.823000
3,GradientBoostingClassifier,72.907489,81.412635,9.623890
4,DecisionTreeClassifier,70.044053,78.656704,0.081050
5,GaussianNB,63.766520,63.456833,0.018884
6,RandomForestClassifier,62.114537,70.770068,1.140600
7,LogisticRegression,47.136564,53.901479,0.441061
8,KNeighborsClassifier,41.519824,42.627665,0.027158


In [18]:
print(classification_report(y_test, pred))
print()
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

         Bus       0.48      0.36      0.41       164
         Car       0.11      0.35      0.16        34
       Still       0.50      0.96      0.66        76
       Train       0.49      0.84      0.62        86
     Walking       0.96      0.66      0.78       548

    accuracy                           0.64       908
   macro avg       0.51      0.63      0.53       908
weighted avg       0.76      0.64      0.67       908


[[ 59  36   1  52  16]
 [  5  12   0  17   0]
 [  1   0  73   1   1]
 [  0  14   0  72   0]
 [ 58  50  72   5 363]]


# Drop car  category 

In [19]:
df_test_1   =df_test[df_test['target']!='Car']
df_train_1  =df_train[df_train['target']!='Car']

In [21]:
print(df_test.shape)
print(df_train.shape)

(908, 14)
(4985, 14)


# How much info reduced 

In [22]:
print(f" After drop Car category we  reduce \n test data {100-df_test_1.shape[0]/df_test.shape[0]*100} % \n train data {100-df_train_1.shape[0]/df_train.shape[0]*100} %")

 After drop Car category we  reduce 
 test data 3.7444933920704813 % 
 train data 22.988966900702096 %


In [23]:
print(df_train_1.shape)
print(df_test_1.shape)

(3839, 14)
(874, 14)


In [24]:
X_=df_train_1.drop('target',axis=1)
y_=df_train_1.target
X_test_=df_test_1.drop('target',axis=1)
y_test_=df_test_1.target

# train data 

In [25]:
x_train_, x_val_, y_train_, y_val_ = train_test_split(X_, y_,test_size=0.4,stratify = y_,random_state=10 )
result_new = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time_ = time.time()
    model.fit(x_train_, y_train_)
    total_time_= time.time() - start_time_
    valid_pred_new = model.predict(x_val_)
    result_new = result_new.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_val_, valid_pred_new)*100,
                              "Bal Acc.": balanced_accuracy_score(y_val_, valid_pred_new)*100,
                              "Time":     total_time_},
                              ignore_index=True)
result_new_ord = result_new.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
result_new_ord.index += 1 
result_new_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[15:38:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,Bal Acc.,Time
1,LGBMClassifier,98.372396,98.330783,0.602918
2,XGBClassifier,98.307292,98.260990,0.804626
3,GradientBoostingClassifier,98.177083,97.958423,3.559644
4,RandomForestClassifier,97.460938,97.402191,0.508297
5,DecisionTreeClassifier,96.484375,96.490002,0.038843
6,KNeighborsClassifier,93.880208,94.015472,0.013182
7,LogisticRegression,78.190104,79.199020,0.128224
8,GaussianNB,74.869792,76.068988,0.011535


# for test 

In [26]:

test = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time_ = time.time()
    model.fit(X_, y_)
    total_time_ = time.time() - start_time_
    pred_test = model.predict(X_test_)
    test = test.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_test_, pred_test)*100,
                              "Bal Acc.": balanced_accuracy_score(y_test_, pred_test)*100,
                              "Time":     total_time_},
                              ignore_index=True)
test_ord = test.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
test_ord.index += 1 
test_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[15:38:12] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,Bal Acc.,Time
1,LGBMClassifier,86.727689,93.676972,0.662900
2,XGBClassifier,85.926773,92.289443,1.415797
3,GradientBoostingClassifier,80.778032,89.595612,5.886485
4,DecisionTreeClassifier,74.370709,86.971178,0.067177
5,RandomForestClassifier,71.624714,80.962632,0.845451
6,GaussianNB,69.565217,78.544874,0.015469
7,KNeighborsClassifier,53.890160,63.141572,0.021664
8,LogisticRegression,51.258581,69.888403,0.195711


In [27]:
print(classification_report(y_test_, pred_test))
print()
print(confusion_matrix(y_test_, pred_test))

              precision    recall  f1-score   support

         Bus       0.45      0.53      0.49       164
       Still       0.49      0.96      0.65        76
       Train       0.57      0.99      0.72        86
     Walking       0.95      0.66      0.78       548

    accuracy                           0.70       874
   macro avg       0.61      0.79      0.66       874
weighted avg       0.78      0.70      0.71       874


[[ 87   1  59  17]
 [  1  73   1   1]
 [  1   0  85   0]
 [104  76   5 363]]


#  The highest accuracy model selected  manual tuning 

In [123]:

best_=LGBMClassifier(boosting_type='gbdt', num_leaves=60, max_depth=6, learning_rate=0.07,
                          n_estimators=300, subsample_for_bin=200000, objective=None, class_weight=None,
                          min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                          subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, 
                          random_state=None, n_jobs=- 1, silent=True, importance_type='split')
model=Pipeline([('preprocess',preprocessor),('classification',best_)])

In [124]:
model.fit(X_,y_)
pred_test_1=model.predict(X_test_)

In [125]:
accuracy_score(y_test_, pred_test_1)*100,

(89.58810068649885,)

In [126]:
print(classification_report(y_test_, pred_test_1))
print()
print(confusion_matrix(y_test_, pred_test_1))

              precision    recall  f1-score   support

         Bus       0.78      0.96      0.86       164
       Still       0.65      0.99      0.78        76
       Train       1.00      1.00      1.00        86
     Walking       0.99      0.85      0.91       548

    accuracy                           0.90       874
   macro avg       0.85      0.95      0.89       874
weighted avg       0.92      0.90      0.90       874


[[158   1   0   5]
 [  0  75   0   1]
 [  0   0  86   0]
 [ 44  40   0 464]]


# Optuna find best parameter for selected model

In [105]:
from sklearn.calibration import CalibratedClassifierCV
import optuna

In [97]:
oth=Pipeline([('preprocess',preprocessor),('classification',GaussianNB())])

In [ ]:
 n_estimators = trial.suggest_int('n_estimators', 2, 20)
max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))

In [118]:
import optuna

def objective(trial):

    boosting_type = trial.suggest_categorical("boosting_type", ["gbdt", "rf"])
    max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
    n_estimators = trial.suggest_int("n_estimators", 100,500)
    learning_rate=trial.suggest_float("learning_rate", 0.001,0.1)

    lg_lgbm = LGBMClassifier( boosting_type=boosting_type, max_depth=max_depth, 
            n_estimators=n_estimators,learning_rate=learning_rate
        )

    score = cross_val_score(lg_lgbm, X_,y_, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))


[I 2021-05-19 17:01:12,300] A new study created in memory with name: no-name-45478aca-a8e9-4d3f-a4e4-82bbbbf59d68
[W 2021-05-19 17:01:12,367] Trial 0 failed, because the objective function returned nan.
[I 2021-05-19 17:01:16,522] Trial 1 finished with value: 0.9718666520067761 and parameters: {'boosting_type': 'gbdt', 'max_depth': 27.69282025085781, 'n_estimators': 207, 'learning_rate': 0.005322060530539006}. Best is trial 1 with value: 0.9718666520067761.
[W 2021-05-19 17:01:16,587] Trial 2 failed, because the objective function returned nan.
[W 2021-05-19 17:01:16,653] Trial 3 failed, because the objective function returned nan.
[I 2021-05-19 17:01:21,814] Trial 4 finished with value: 0.9846317516940317 and parameters: {'boosting_type': 'gbdt', 'max_depth': 10.861433368507354, 'n_estimators': 373, 'learning_rate': 0.0586916179235361}. Best is trial 4 with value: 0.9846317516940317.
[I 2021-05-19 17:01:22,221] Trial 5 finished with value: 0.8317242067370342 and parameters: {'boosting

[W 2021-05-19 17:01:42,679] Trial 62 failed, because the objective function returned nan.
[W 2021-05-19 17:01:42,730] Trial 63 failed, because the objective function returned nan.
[W 2021-05-19 17:01:42,778] Trial 64 failed, because the objective function returned nan.
[W 2021-05-19 17:01:42,828] Trial 65 failed, because the objective function returned nan.
[W 2021-05-19 17:01:42,880] Trial 66 failed, because the objective function returned nan.
[W 2021-05-19 17:01:42,925] Trial 67 failed, because the objective function returned nan.
[W 2021-05-19 17:01:42,971] Trial 68 failed, because the objective function returned nan.
[W 2021-05-19 17:01:43,025] Trial 69 failed, because the objective function returned nan.
[W 2021-05-19 17:01:43,083] Trial 70 failed, because the objective function returned nan.
[W 2021-05-19 17:01:43,138] Trial 71 failed, because the objective function returned nan.
[W 2021-05-19 17:01:43,185] Trial 72 failed, because the objective function returned nan.
[W 2021-05

Accuracy: 0.985412798084441
Best hyperparameters: {'boosting_type': 'gbdt', 'max_depth': 4.461224349311469, 'n_estimators': 305, 'learning_rate': 0.06753369486128141}


# With optuna best parameter

 the best accuracy 89.8  in  max_depth= 5, n_estimators= 290, learning_rate= 0.069

In [225]:

lgb_af_=LGBMClassifier( max_depth= 5, n_estimators= 300, learning_rate= 0.069)
model_end=Pipeline([('preprocess',preprocessor),('classification',lgb_af_)])

In [226]:
model_end.fit(X_,y_)
pred_test_end=model_end.predict(X_test_)

In [227]:
print(f"Accuracy {accuracy_score(y_test_, pred_test_end)*100}")
print(classification_report(y_test_, pred_test_end))
print()
print(confusion_matrix(y_test_, pred_test_end))

Accuracy 89.81693363844394
              precision    recall  f1-score   support

         Bus       0.83      0.95      0.89       164
       Still       0.60      0.99      0.74        76
       Train       0.99      1.00      0.99        86
     Walking       0.99      0.85      0.92       548

    accuracy                           0.90       874
   macro avg       0.85      0.95      0.88       874
weighted avg       0.93      0.90      0.90       874


[[156   2   1   5]
 [  1  75   0   0]
 [  0   0  86   0]
 [ 31  49   0 468]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3e7089f3-11a5-48ad-89c4-39a166311a14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>